In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import keras 
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from PIL import Image
import matplotlib.image as mpg
import seaborn as sns
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'sans-serif' 
plt.rcParams['font.serif'] = 'Ubuntu' 
plt.rcParams['font.monospace'] = 'Ubuntu Mono' 
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.labelsize'] = 12 
plt.rcParams['axes.labelweight'] = 'bold' 
plt.rcParams['axes.titlesize'] = 12 
plt.rcParams['xtick.labelsize'] = 12 
plt.rcParams['ytick.labelsize'] = 12 
plt.rcParams['legend.fontsize'] = 12 
plt.rcParams['figure.titlesize'] = 12 
plt.rcParams['image.cmap'] = 'jet' 
plt.rcParams['image.interpolation'] = 'none' 
plt.rcParams['figure.figsize'] = (10, 10
                                 ) 
plt.rcParams['axes.grid']=True
plt.rcParams['lines.linewidth'] = 2 
plt.rcParams['lines.markersize'] = 8
colors = ['xkcd:pale orange', 'xkcd:sea blue', 'xkcd:pale red', 'xkcd:sage green', 'xkcd:terra cotta', 'xkcd:dull purple', 'xkcd:teal', 'xkcd: goldenrod', 'xkcd:cadet blue',
'xkcd:scarlet']
bbox_props = dict(boxstyle="round,pad=0.3", fc=colors[0], alpha=.5)

In [ ]:
data = pd.read_csv('D://POC//ComputerVisionSolution//Surface_defects_detection//archive//age_gender.csv')

In [ ]:
# data = data.sample(frac=1).reset_index().loc[0:1000]
data

In [ ]:
def Convert(string):
    li = list(string.split(" "))
    return li
  

In [ ]:
PIXELS=[]
for i in range(len(data)):
    PIXELS.append(Convert(data.pixels[i]))

In [ ]:
NEW_PIXELS = []
for p in range(len(PIXELS)):
    new_pixels = []
    for q in range(len(PIXELS[p])):
        new_pixels.append(int(PIXELS[p][q]))
    NEW_PIXELS.append(np.array(new_pixels).reshape((48,48,1)))

In [ ]:
images = np.array(NEW_PIXELS)

In [ ]:
data = data.drop(columns=['index','img_name'])

In [ ]:
sns.countplot(data.gender,palette='plasma')
plt.xticks([0,1],['Male','Female'])
plt.grid(True)

In [ ]:
data.pixels = NEW_PIXELS

In [ ]:
for i in range(1,5):
    J = np.random.choice(np.arange(0,1000,1))
    plt.subplot(2,2,i)
    plt.title('Label : '+ str(data.gender[J]),fontsize=20)
    plt.imshow(np.array(data.pixels[J]).reshape((48,48)),cmap='gray')
    plt.tight_layout()

In [ ]:

classifier = Sequential()
size = 48
# Step 1 - Convolution
classifier.add(Conv2D(64, (2, 2), input_shape = (size,size, 1), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(64, (2, 2), input_shape = (size,size, 1), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())

#classifier.add(Dense(units = 32, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.summary()

In [ ]:
labels = np.array(data.gender)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.15)

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
history = classifier.fit(train_images, train_labels, epochs=10, 
                validation_data=(test_images, test_labels),batch_size=10)

In [ ]:
loss = history.history['accuracy']
val_loss = history.history['val_accuracy']

In [ ]:
plt.plot(np.arange(1,len(loss)+1,1),loss,color='navy', label = 'Accuracy')
plt.plot(np.arange(1,len(loss)+1,1),val_loss,color='red',label='Validation Accuracy')
plt.legend(fontsize=15)

In [ ]:
plt.plot(classifier.predict(test_images),'.',color='red',label='Predicted Probabilty')
plt.plot(test_labels,'.',color='navy',label='Actual Labels')
plt.xlabel('Instance Number')
plt.ylabel('Probability')
plt.legend()

In [ ]:
predictions = classifier.predict(test_images)

In [ ]:
decision = []

for p in predictions:
    if p>=0.5:
        decision.append(1)
    else:
        decision.append(0)

In [ ]:
sns.heatmap(confusion_matrix(decision,test_labels),cmap='plasma',annot=True,annot_kws={"size": 32})
plt.xticks([0.50,1.50],['Male','Female'],fontsize=20)
plt.yticks([0.50,1.50],['Male','Female'],fontsize=20)

In [ ]:
clf_report = classification_report(decision,test_labels,
                                   labels=[0,1],
                                   target_names=['Male','Female'],
                                   output_dict=True)

In [ ]:
plt.subplot(2,1,1)
sns.heatmap(pd.DataFrame(clf_report).iloc[:-2, :-3].T, annot=True)
plt.subplot(2,1,2)
sns.heatmap(pd.DataFrame(clf_report).iloc[:-2, 3:].T, annot=True)
plt.tight_layout()

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
def ImagePreProcessing(image_path):
    img = Image.open(image_path)
    img = img.resize((48,48))
    img_array = np.asarray(img)
    baw_img = rgb2gray(img_array).astype(int)
    final_img = baw_img.reshape((48,48,1))
    return final_img

In [ ]:
plt.imshow(ImagePreProcessing('test.JPG'),cmap='gray')

In [ ]:
image_test = ImagePreProcessing('test.JPG')

In [ ]:
prob = 1-classifier.predict(np.array([image_test]))[0][0]

In [ ]:
print("I'm a man, and the classifier says that I'm a man with probability %.2f" %(prob*100) + '%')

In [ ]:
model = keras.models.save_model(classifier,'/Users/pierohmd/Desktop/archive/classifier')

In [ ]:
model = keras.models.load_model('/Users/pierohmd/Desktop/archive/classifier')

In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras
visualkeras.layered_view(model=classifier, to_file ='GenderClassifier.png')

In [54]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# from keras.utils import to_categorical
from tqdm import tqdm
from sqlite_db_connect import SQLiteConnect
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [55]:
class Classifier(SQLiteConnect):
    def ___init__(self):
        super.__init__()


In [56]:
classifierObj = Classifier()
data = classifierObj.getRecords('SELECT FileName,FilePath,Tag from RESOURCE_DETAILS ')
train = pd.DataFrame(data)

Database created and Successfully Connected to SQLite
query-> SELECT FileName,FilePath,Tag from RESOURCE_DETAILS  rec [{'FileName': 'cast_def_0_65.jpeg', 'FilePath': 'Projects\\1\\1\\cast_def_0_65.jpeg', 'Tag': 'notdefected'}, {'FileName': 'cast_def_0_67.jpeg', 'FilePath': 'Projects\\1\\1\\cast_def_0_67.jpeg', 'Tag': 'defected'}, {'FileName': 'cast_def_0_85.jpeg', 'FilePath': 'Projects\\1\\1\\cast_def_0_85.jpeg', 'Tag': 'defected'}, {'FileName': 'cast_def_0_87.jpeg', 'FilePath': 'Projects\\1\\1\\cast_def_0_87.jpeg', 'Tag': 'notdefected'}, {'FileName': 'cast_def_0_88.jpeg', 'FilePath': 'Projects\\1\\1\\cast_def_0_88.jpeg', 'Tag': 'notdefected'}, {'FileName': 'cast_def_0_108.jpeg', 'FilePath': 'Projects\\1\\1\\cast_def_0_108.jpeg', 'Tag': 'defected'}, {'FileName': 'cast_def_0_112.jpeg', 'FilePath': 'Projects\\1\\1\\cast_def_0_112.jpeg', 'Tag': 'defected'}, {'FileName': 'cast_def_0_118.jpeg', 'FilePath': 'Projects\\1\\1\\cast_def_0_118.jpeg', 'Tag': 'defected'}, {'FileName': 'cast_def_0_1

In [83]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
image_shape=(100,100,1)
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(os.path.join('uploads',train['FilePath'][i]), target_size=image_shape , color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 82/82 [00:00<00:00, 88.64it/s]


In [84]:
label_encoder = LabelEncoder()
dy=train['Tag'].values
vec = label_encoder.fit_transform(dy)
y = to_categorical(vec)
output_classes= len(set(vec))

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [76]:
model_save_path = os.path.join(os.getcwd(),'uploads','User_Model_Files','mynewmodel.hdf5')
early_stop = EarlyStopping(monitor='val_loss',patience=5)
checkpoint = ModelCheckpoint(filepath=model_save_path, verbose=1, save_best_only=True, monitor='val_loss')

In [77]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=image_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [67]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(7,7), strides=2, input_shape=image_shape, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=1, input_shape=image_shape, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=1, input_shape=image_shape, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(units=224, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=output_classes, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [71]:
model = Sequential()
# Step 1 - Convolution
model.add(Conv2D(64, (2, 2), input_shape = image_shape, activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (2, 2), input_shape = (size,size, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = output_classes, activation = 'sigmoid'))
# Compiling the CNN
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_62 (Conv2D)           (None, 27, 27, 64)        320       
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 12, 12, 64)        16448     
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 2304)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 32)                73760     
_________________________________________________________________
dense_41 (Dense)             (None, 3)               

In [72]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test),callbacks=[early_stop,checkpoint])
# model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test) , callbacks=[checkpoint]);

Epoch 1/50
3/3 [==============================] - 2s 349ms/step - loss: 0.6830 - accuracy: 0.4301 - val_loss: 0.6310 - val_accuracy: 0.2353

Epoch 00001: val_loss did not improve from 0.39861
Epoch 2/50
3/3 [==============================] - 0s 34ms/step - loss: 0.6127 - accuracy: 0.5428 - val_loss: 0.5529 - val_accuracy: 0.7647

Epoch 00002: val_loss did not improve from 0.39861
Epoch 3/50
3/3 [==============================] - 0s 36ms/step - loss: 0.5621 - accuracy: 0.4692 - val_loss: 0.4912 - val_accuracy: 0.7647

Epoch 00003: val_loss did not improve from 0.39861
Epoch 4/50
3/3 [==============================] - 0s 36ms/step - loss: 0.5227 - accuracy: 0.4341 - val_loss: 0.4787 - val_accuracy: 0.4706

Epoch 00004: val_loss did not improve from 0.39861
Epoch 5/50
3/3 [==============================] - 0s 45ms/step - loss: 0.4991 - accuracy: 0.6048 - val_loss: 0.4505 - val_accuracy: 0.7647

Epoch 00005: val_loss did not improve from 0.39861
Epoch 6/50
3/3 [============================

In [90]:
from tensorflow import keras
model1 = keras.models.load_model(os.path.join(os.getcwd(),'uploads\\User_Model_Files\\classification demo-9.hdf5'))

In [91]:
model1.predict_classes(X_test)

d:\python\ttl\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0], dtype=int64)